In [20]:
import time
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

# 特征处理

In [21]:
df = pd.read_csv(r'.././Data/TRAINSET_STOCK.csv')
df_add = pd.read_csv(r'.././Data/20190506_STOCK.csv')
df_test = pd.read_csv(r'.././Data/20190514_STOCK.csv')
df = pd.concat([df,df_add,df_test])
df['vol_change'] = df.groupby(["ts_code"])['vol'].pct_change()
df['low_change'] = df.groupby(["ts_code"])['low'].pct_change()
df['high_change'] = df.groupby(["ts_code"])['high'].pct_change()
df['vol_change'] = df['vol_change'].fillna(0)
df['low_change'] = df['vol_change'].fillna(0)
df['high_change'] = df['vol_change'].fillna(0)
df =  df[['ts_code','trade_date','name','pct_change','vol_change','high_change','low_change','y']]
# l=0
# for arr in df['ts_code']:
#   df.loc[(df['ts_code'] ==arr) == True, '行业'] = l
#   l=l+1


In [22]:
df = df[['pct_change','vol_change','high_change','low_change','y']]
df.head()

,pct_change,vol_change,high_change,low_change,y
0,1.33,0.000000,0.000000,0.000000,1
1,0.19,0.039698,0.039698,0.039698,1
2,-0.39,-0.134904,-0.134904,-0.134904,0
3,0.74,-0.087730,-0.087730,-0.087730,1
4,0.51,0.222145,0.222145,0.222145,1


# LSTM

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,Activation
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

# 没有交叉验证


In [24]:
dataset = df.values
dataset

array([[ 1.33      ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.19      ,  0.03969754,  0.03969754,  0.03969754,  1.        ],
       [-0.39      , -0.13490358, -0.13490358, -0.13490358,  0.        ],
       ...,
       [-0.83      , -0.07570617, -0.07570617, -0.07570617,  0.        ],
       [-0.64      , -0.03678727, -0.03678727, -0.03678727,  0.        ],
       [-0.54      , -0.01516826, -0.01516826, -0.01516826,  0.        ]])

In [25]:
x = []
y = []
for i in range(len(dataset)-15):
    x.append(dataset[i:(i+10),:5].tolist())
    y.append(dataset[(i+10):(i+15),4,np.newaxis].tolist())
xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=0)
xtrain, xtest, ytrain, ytest = np.array(xtrain),np.array(xtest),np.array(ytrain),np.array(ytest)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape
# xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1]*xtrain.shape[2], 1))
# xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[1], 1))
ytrain = np.reshape(ytrain, (ytrain.shape[0], ytrain.shape[1]))
ytest= np.reshape(ytest, (ytest.shape[0], ytest.shape[1]))
xtrain.shape


(32056, 10, 5)

In [26]:
regressor = Sequential()
regressor.add(LSTM(units = 60, return_sequences = True, input_shape = (xtrain.shape[1], 5)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60,return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 5))
regressor.add(Activation('sigmoid'))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(xtrain, ytrain, epochs = 100, batch_size = 50)

Epoch 1/100
32056/32056 [==============================] - 34s 1ms/step - loss: 0.2479
Epoch 2/100
32056/32056 [==============================] - 30s 936us/step - loss: 0.2457
Epoch 3/100
32056/32056 [==============================] - 30s 947us/step - loss: 0.2420
Epoch 4/100
32056/32056 [==============================] - 34s 1ms/step - loss: 0.2374
Epoch 5/100
32056/32056 [==============================] - 32s 1ms/step - loss: 0.2325
Epoch 6/100
32056/32056 [==============================] - 34s 1ms/step - loss: 0.2277
Epoch 7/100
32056/32056 [==============================] - 38s 1ms/step - loss: 0.2232
Epoch 8/100
32056/32056 [==============================] - 39s 1ms/step - loss: 0.2182
Epoch 9/100
32056/32056 [==============================] - 32s 1ms/step - loss: 0.2135
Epoch 10/100
32056/32056 [==============================] - 42s 1ms/step - loss: 0.2082
Epoch 11/100
32056/32056 [==============================] - 34s 1ms/step - loss: 0.2032
Epoch 12/100
32056/32056 [===========

Epoch 93/100
32056/32056 [==============================] - 28s 873us/step - loss: 0.0985
Epoch 94/100
32056/32056 [==============================] - 2160s 67ms/step - loss: 0.0982
Epoch 95/100
32056/32056 [==============================] - 27s 852us/step - loss: 0.0981
Epoch 96/100
32056/32056 [==============================] - 4640s 145ms/step - loss: 0.0977
Epoch 97/100
32056/32056 [==============================] - 31s 963us/step - loss: 0.0971
Epoch 98/100
32056/32056 [==============================] - 32s 1ms/step - loss: 0.0970
Epoch 99/100
32056/32056 [==============================] - 29s 919us/step - loss: 0.0966
Epoch 100/100
32056/32056 [==============================] - 30s 925us/step - loss: 0.0968


In [27]:
predicted_stock_price = regressor.predict(xtest)
a = []
for i in range(len(ytest)):
    fpr, tpr, thresholds = roc_curve(ytest[i], predicted_stock_price[i], pos_label=1)
    auc(fpr, tpr)
    a.append(auc(fpr, tpr))
a=pd.DataFrame(a)
a=a.dropna()
a.mean()

0    0.898177
dtype: float64

# 预测结果

In [49]:
df_test1= pd.read_csv(r'.././Data/20190506_STOCK.csv')
df_test2 = pd.read_csv(r'.././Data/20190514_STOCK.csv')
df_test3 = pd.read_csv(r'.././Data/20190521_STOCK.csv')
df_test4 = pd.concat([df_test1,df_test2,df_test3])
# df_test4.iloc[1:3]
df_test4.head()
    


,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y
0,801010,20190402,农林牧渔,3605.26,3485.11,3605.26,3513.06,-41.36,-1.16,236389.0,2761598.0,39.92,3.78,0
1,801020,20190402,采掘,2935.97,2905.01,2941.36,2935.37,22.18,0.76,282181.0,1732160.0,15.25,1.34,1
2,801030,20190402,化工,2971.25,2961.67,3011.61,2996.91,55.41,1.88,762659.0,7911222.0,18.94,2.06,1
3,801040,20190402,钢铁,2743.16,2711.59,2743.16,2732.47,0.80,0.03,195360.0,1062437.0,7.85,1.23,1
4,801050,20190402,有色金属,3195.75,3152.60,3261.40,3234.95,63.39,2.00,582889.0,5159532.0,32.45,2.26,1


In [50]:
df_test4['vol_change'] = df_test4.groupby(["ts_code"])['vol'].pct_change()
df_test4['low_change'] = df_test4.groupby(["ts_code"])['low'].pct_change()
df_test4['high_change'] = df_test4.groupby(["ts_code"])['high'].pct_change()
df_test4 =  df_test4[['ts_code','trade_date','name','pct_change','vol_change','high_change','low_change','y']]


# 回测5.08-5.14得分

In [228]:
df_test4.trade_date
index1=(df_test4.trade_date > 20190507)
df_huice_target = df_test4[index1]

In [229]:
index2= (df_test4.trade_date > 20190418)&(df_test4.trade_date < 20190508)
df_huice = df_test4[index2]

In [230]:
df_huice_target1=df_huice_target[(df_huice_target.trade_date==20190508)][['name']]
colums1=[20190508,20190509,20190510,20190513,20190514]
for arr1 in colums1:
    a=df_huice_target[(df_huice_target.trade_date==arr1)][['name','y']]
    a.columns=['name',arr1]
    df_huice_target1=df_huice_target1.merge(a,how='left',on='name')


In [231]:
df_huice_pr=pd.DataFrame()
df_huice_pr['trade_date']=[20190508,20190509,20190510,20190513,20190514]
for arr in colums:
    df4 = df_huice.loc[df_huice['name']==arr]
    dataset4=df4.values
    x4=dataset4[0:10,3:8].tolist()
    x4=np.array(x4)
    x4=np.reshape(x4, (1,x4.shape[0],x4.shape[1]))
    ytest4=regressor.predict(x4)
    y_p4=pd.DataFrame(ytest4)
    y_p4=y_p4.T
    y_p4['trade_date']=[20190508,20190509,20190510,20190513,20190514]
    y_p4.columns=[arr,'trade_date']
    df_huice_pr = df_huice_pr.merge(y_p4,how='left',on='trade_date')
df_huice_pr

,trade_date,农林牧渔,采掘,化工,钢铁,有色金属,电子,家用电器,食品饮料,纺织服装,...,电气设备,国防军工,计算机,传媒,通信,银行,非银金融,汽车,机械设备,交银装备
0,20190508,0.537676,0.465144,0.547662,0.390392,0.809716,0.501885,0.967549,0.874385,0.594427,...,0.315485,0.952320,0.246173,0.959684,0.900816,0.501268,0.796650,0.205582,0.395713,0.349191
1,20190509,0.967028,0.810550,0.280799,0.428282,0.700356,0.128405,0.380407,0.243948,0.264333,...,0.234432,0.475878,0.245181,0.800604,0.262624,0.869332,0.131206,0.810751,0.581561,0.371339
2,20190510,0.014620,0.039050,0.859187,0.809891,0.974666,0.391452,0.050880,0.418528,0.908377,...,0.503303,0.989166,0.581806,0.606178,0.220639,0.775498,0.426956,0.094474,0.791969,0.637631
3,20190513,0.585662,0.151758,0.668108,0.588080,0.955087,0.823792,0.015348,0.045544,0.709630,...,0.436818,0.985061,0.327000,0.514012,0.336745,0.964445,0.047425,0.059912,0.803821,0.876094
4,20190514,0.863155,0.960605,0.958759,0.792201,0.874684,0.943877,0.659429,0.142347,0.908666,...,0.971231,0.986006,0.988236,0.939456,0.836124,0.566570,0.335100,0.959068,0.879734,0.897793


In [232]:
df_huice_pr.index=df_huice_pr['trade_date']
df_huice_pr=df_huice_pr.drop('trade_date',axis=1)

In [233]:
df_huice_target1

,name,20190508,20190509,20190510,20190513,20190514
0,农林牧渔,0,1,1,0,0
1,采掘,0,0,1,0,0
2,化工,0,0,1,0,0
3,钢铁,0,0,1,0,0
4,有色金属,1,0,1,0,1
5,电子,0,0,1,0,0
6,家用电器,0,0,1,0,0
7,食品饮料,0,0,1,1,0
8,纺织服装,1,0,1,1,0
9,轻工制造,1,0,1,0,0


In [234]:
toal=0
df_huice_pr1=df_huice_pr.T
df_huice_pr1.index=range(34)
for arr2 in df_huice_pr1.columns:
    f, t, t1 = roc_curve(df_huice_target1[arr2],df_huice_pr1[arr2],pos_label=1)
    toal=toal+auc(f, t)
    print(arr2,auc(f,t))
print('mean',toal/5)   

20190508 0.5586206896551724
20190509 1.0
20190510 nan
20190513 0.328125
20190514 0.6129032258064516
mean nan


# 预测5.22-5.28

In [68]:
# 行业名字和行业代码的对应
df_me=pd.DataFrame(df_test4[['ts_code','name']])
df_me=df_me.drop_duplicates()
df_me
df8=df_me[['name']]
df8=df8.T
df8.values

array([['农林牧渔', '采掘', '化工', '钢铁', '有色金属', '电子', '家用电器', '食品饮料', '纺织服装',
        '轻工制造', '医药生物', '公用事业', '交通运输', '房地产', '商业贸易', '休闲服务', '综合',
        '申万制造', '申万消费', '申万投资', '申万服务', '申万300指数', '建筑材料', '建筑装饰',
        '电气设备', '国防军工', '计算机', '传媒', '通信', '银行', '非银金融', '汽车', '机械设备',
        '交银装备']], dtype=object)

In [64]:
# 预测未来五天的结果
colums=['农林牧渔', '采掘', '化工', '钢铁', '有色金属', '电子', '家用电器', '食品饮料', '纺织服装',
        '轻工制造', '医药生物', '公用事业', '交通运输', '房地产', '商业贸易', '休闲服务', '综合',
        '申万制造', '申万消费', '申万投资', '申万服务', '申万300指数', '建筑材料', '建筑装饰',
        '电气设备', '国防军工', '计算机', '传媒', '通信', '银行', '非银金融', '汽车', '机械设备',
        '交银装备']
# colums=[801010, 801020, 801030, 801040, 801050, 801080, 801110, 801120,
#         801130, 801140, 801150, 801160, 801170, 801180, 801200, 801210,
#         801230, 801250, 801260, 801270, 801280, 801300, 801710, 801720,
#         801730, 801740, 801750, 801760, 801770, 801780, 801790, 801880,
#         801890, 802600]
range_index = (df_test4.trade_date > 20190507)
df_final = df_test4[range_index]
df_pr=pd.DataFrame()
for arr in colums:
    df3 = df_final.loc[df_final['name']==arr]
    dataset2=df3.values
    x1=dataset2[0:10,3:8].tolist()
    x1=np.array(x1)
    x1=np.reshape(x1, (1,x1.shape[0],x1.shape[1]))
    ytest1=regressor.predict(x1)
    y_p=pd.DataFrame(ytest1)
    y_p=y_p.T
    y_p['name']=arr
    y_p['trade_date']=[20190522,20190523,20190524,20190527,20190528]
    df_pr = pd.concat([df_pr,y_p])
df_pr=df_me.merge(df_pr,how='left',on='name')
df_pr.columns=['ts_code','name','p','trade_date']
df_pr=df_pr[['ts_code','trade_date','p']]
df_pr
# df_pr.to_csv('./结果提交.csv')

,ts_code,trade_date,p
0,801010,20190522,0.841898
1,801010,20190523,0.006166
2,801010,20190524,0.974308
3,801010,20190527,0.025921
4,801010,20190528,0.994883
5,801020,20190522,0.069095
6,801020,20190523,0.933595
7,801020,20190524,0.833302
8,801020,20190527,0.926811
9,801020,20190528,0.022047


# 交叉验证

In [28]:
kf = KFold(n_splits=10, shuffle=True, random_state=520)

In [ ]:
i=0
for train,test in kf.split(xtrain):
    a = []
    b= []
    X_train, X_valid= xtrain[train,:], xtrain[test,:]
    y_train, y_valid = ytrain[train,:], ytrain[test,:]
    print(i, 'fold')
    model=regressor.fit(X_train,y_train, epochs = 100, batch_size = 50)
    predicted_stock_price1 = regressor.predict(X_valid)
    for j in range(len(y_valid)):
        fpr1, tpr1, thresholds1 = roc_curve(y_valid[j],predicted_stock_price1[j], pos_label=1)
        a.append(auc(fpr1, tpr1))
    a=pd.DataFrame(a)
    a=a.dropna()
    print(a.mean())
    predicted_stock_price2 = regressor.predict(xtest)
    for k in range(len(ytest)):
        fpr2, tpr2, thresholds2 = roc_curve(ytest[k], predicted_stock_price2[k], pos_label=1)
        b.append(auc(fpr2, tpr2))
    b=pd.DataFrame(b)
    b=b.dropna()
    print(b.mean())
    i=i+1

0 fold
[[1. 0. 1. 0. 1.]
 [0. 1. 1. 1. 1.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 1. 1. 0. 1.]
 [1. 1. 1. 1. 0.]
 [1. 1. 1. 0. 1.]]
Epoch 1/100
28850/28850 [==============================] - 34s 1ms/step - loss: 0.2428
Epoch 2/100
28750/28850 [============================>.] - ETA: 0s - loss: 0.2395

# 新闻

In [0]:
df1 = pd.read_csv(r'./TRAINSET_STOCK.csv')
df_add = pd.read_csv(r'./20190506_STOCK.csv')
df_test = pd.read_csv(r'./20190514_STOCK.csv')
df1 = pd.concat([df1,df_add,df_test])

In [84]:
df2=df1[['name','ts_code']]
df2 = df2.drop_duplicates(keep='first')


,0,1152,2304,3456,3656,4608,5760,6912,8064,9216,10368,11520,12672,13824,14976,16128,17280,17480,18432,19584,20736,21888,23040,24192,25344,26496,27648,28800,29952,31104,32256,33408,34560,35712,36864,38016
name,农林牧渔,采掘,化工,黑色金属,钢铁,有色金属,电子,家用电器,食品饮料,纺织服装,轻工制造,医药生物,公用事业,交通运输,房地产,商业贸易,餐饮旅游,休闲服务,综合,申万制造,申万消费,申万投资,申万服务,申万300指数,建筑材料,建筑装饰,电气设备,国防军工,计算机,传媒,通信,银行,非银金融,汽车,机械设备,交银装备
ts_code,801010,801020,801030,801040,801040,801050,801080,801110,801120,801130,801140,801150,801160,801170,801180,801200,801210,801210,801230,801250,801260,801270,801280,801300,801710,801720,801730,801740,801750,801760,801770,801780,801790,801880,801890,802600


In [86]:
df_news1 = pd.read_csv(r'./TRAINSET_NEWS.csv')
df_news2= pd.read_csv(r'./20190506_NEWS.csv')
df_news3= pd.read_csv(r'./20190514_NEWS.csv')
df_news = pd.concat([df_news1,df_news2,df_news3])
df_news.info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27077 entries, 0 to 112
Data columns (total 4 columns):
id         27077 non-null object
date       27077 non-null int64
title      27048 non-null object
content    26915 non-null object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


# Tensorflow

In [0]:
import tensorflow as tf
time_step=10      #时间步
rnn_unit=25       #hidden layer units
batch_size=80     #每一批次训练多少个样例
input_size=3      #输入层维度
output_size=1     #输出层维度
lr=0.0006 


In [0]:
# 训练集
def get_train_data(batch_size=80,time_step=10,train_begin=0,train_end=30000):
    batch_index=[]
    data_train=dataset[train_begin:train_end]
    normalized_train_data=(data_train-np.mean(data_train,axis=0))/np.std(data_train,axis=0)  #标准化
    train_x,train_y=[],[]   #训练集x和y初定义
    for i in range(len(normalized_train_data)-time_step):
        if i % batch_size==0:
            batch_index.append(i)
            x=normalized_train_data[i:i+time_step,:3]
            y=normalized_train_data[i:i+time_step,2,np.newaxis]
            train_x.append(x.tolist())
            train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data)-time_step))
    return batch_index,train_x,train_y
 


In [0]:
# 测试集
def get_test_data(time_step=10,test_begin=30000):
    data_test=dataset[test_begin:]
    mean=np.mean(data_test,axis=0)
    std=np.std(data_test,axis=0)
    normalized_test_data=(data_test-mean)/std  #标准化
    size=(len(normalized_test_data)+time_step-1)//time_step  #有size个sample 
    test_x,test_y=[],[]  
    for i in range(size-1):
        x=normalized_test_data[i*time_step:(i+1)*time_step,:3]
        y=normalized_test_data[i*time_step:(i+1)*time_step,2]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i+1)*time_step:,:3]).tolist())
    test_y.extend((normalized_test_data[(i+1)*time_step:,2]).tolist())
    return mean,std,test_x,test_y

In [0]:
# 模型搭建
weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[rnn_unit, ])),
    'out': tf.Variable(tf.constant(0.1, shape=[1, ]))
}
def lstm(X):     
    batch_size=tf.shape(X)[0]
    time_step=tf.shape(X)[1]
    w_in=weights['in']
    b_in=biases['in']  
    input=tf.reshape(X,[-1,input_size])  #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(input,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch_size,dtype=tf.float32)
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out
    return pred,final_states

In [0]:
def train_lstm(batch_size=80,time_step=10,train_begin=0,train_end=30000):
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    Y=tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    batch_index,train_x,train_y=get_train_data(batch_size,time_step,train_begin,train_end)
    pred,_=lstm(X)
    #损失函数
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
    saver=tf.train.Saver(tf.global_variables(),max_to_keep=15)
    module_file = tf.train.latest_checkpoint()    
    with tf.Session() as sess:
        #sess.run(tf.global_variables_initializer())
        saver.restore(sess, module_file)
        #重复训练2000次
        for i in range(2000):
            for step in range(len(batch_index)-1):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[batch_index[step]:batch_index[step+1]],Y:train_y[batch_index[step]:batch_index[step+1]]})
            print(i,loss_)
            if i % 200==0:
                print("保存模型：",saver.save(sess,'stock2.model',global_step=i))